# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [43]:
# imports
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [44]:
# constants
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
OLLAMA_API = "http://localhost:11434/api/chat"
openai = OpenAI()
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


In [48]:
system_prompt = """You are an expert of both Python and LLM engineering.\n
                Please answer question to your mentee in fluent Korean."""


def user_prompt(query):
    user_prompt = f"answer this question technically:\n"
    user_prompt += query
    #user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt
    
# messages
def create_answer(query, MODEL):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt(query)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))
# here is the question; type over this to ask something new

query = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""


In [51]:
# Get gpt-4o-mini to answer, with streaming,MD
print(create_answer(query,MODEL_GPT))

이 코드는 파이썬에서 `yield from` 구문과 집합 내포(set comprehension)를 사용하여 특정 작업을 수행합니다. 아래에 이 코드의 기능과 이유를 설명하겠습니다.

### 코드 설명
```python
yield from {book.get("author") for book in books if book.get("author")}
```

1. **집합 내포**:
   - `{book.get("author") for book in books if book.get("author")}` 부분은 집합 내포입니다. 이는 `books`라는 iterable(예: 리스트)에서 각 `book` 객체를 순회하며, `book`의 `"author"` 키에 해당하는 값을 가져옵니다.
   - `if book.get("author")` 조건은 `book`에서 `"author"` 키의 값이 존재하는 경우에만 포함되도록 하여, `None` 또는 빈 문자열과 같은 falsy 값은 걸러냅니다. 결국, 이 집합 내포는 `books` 목록에 있는 모든 저자 이름의 고유한 집합을 생성합니다.

2. **`yield from` 구문**:
   - `yield from`은 파이썬 제너레이터의 기능으로, 특정 이터러블(여기서는 집합)에서 값을 순차적으로 반환합니다. 즉, 이 코드에서는 위에서 생성된 저자 이름의 집합에서 각 저자 이름을 하나씩 반환합니다.

### 요약
- 이 코드는 `books`라는 리스트에서 각 책의 저자를 추출하고, 저자가 존재하는 경우에만 고유한 저자 이름의 집합을 만들며, 그 집합의 저자 이름들을 순차적으로 반환하려는 목적을 가지고 있습니다.
- 따라서 이 코드의 주 목적은 책들의 저자 정보를 효율적으로 수집하고 반환하는 것입니다.

None


In [49]:

    
def create_answer_llama(query, MODEL):
    #website = Website(url)
    response = ollama_via_openai.chat.completions.create(
        model = MODEL,
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt(query)}
          ],)
    result = response.choices[0].message.content
    display(Markdown(result))
    

In [50]:
# Get Llama 3.2 to answer
print(create_answer_llama(query,MODEL_LLAMA))

이 코드는 다음 두 가지 Things를 करत다.

### 1. 데이터가 나와서 반복하는 Mechanism을 구현한다
`yield from ...`이라는 keyword는 yield 연 overload을 implements하다.

반면 일반적인 `yield` keyword처럼, 만약 예외가 되게된다면, 그 전까지의 연역에서 yielded한 value를 제거할 수 있게 한다. `yield from`,로gether는, yielded from 이 preceding value를 제거한다.

```python
def generator_func():
    yield 0
    try:
        yield 1
    except Exception as e:
        print(e) #Exception이 발생하면 0을 제거하고에러메세지를rint하게 한다.
```
이렇게 작동할 수 있다:

- `gen = generator_func()`

```python
print(next(gen)) == 0
# True
```

```python
try:
    next(gen)
except ValueError:
    print("False")
## True
```

###2. 예시로 사용할 예를들어 보자
이 경우, books object의 author field가 있는 book을 only select하고 yield from 해서 이 books object를 반복하기위한 generator function을 만든다.

```python
books = [
    {"title": "Book1", "author": "AuthorA"},
    {"title": ""}, # null인 book이 있다.
    {
        "title":"Book3","author":"AuthorC"
    },
    {"title":""},
{"title":"Book5","author':'AuthorE'} #author field가 presence가 있는 book
]

books_gen = (book["author"] for book in books if book.get("author"))
```
`next(next(books_gen))`,
`...`으로 book의 author field가 existence하는 every time, 
한개씩 book object를 iteration시킬 수 있다.
`next(...)`는 nextgenator의 value를 next iteration 시키지 않도록 한다.
이 thereby, books object의 author field가 presence가 있는 각 book을 iteratation시킬 수 있게 한다.


`yield from ...`,로gether는 yield from which preceding value가 exception을 발생시켜서 제거하지 못한다. 
in this case

```python
books_gen = (book["author"] for book in books if book.get("author"))
exception_gen = (1) # generator class.
next_value= next(books_gen)
print(next_value) == 'AuthorA'
print(next_exception()) # exception value가 none 

# books_gen과 exception_gen between 2는 different

```

None
